In [1]:
import pandas

df = pandas.read_csv("conxref_object_link.csv")
df.head()

,s,object,Unnamed: 2
0,"""https://example.com/ConXrefs/156404""","""https://example.com/Objects/84043""",NaN
1,"""https://example.com/ConXrefs/129378""","""https://example.com/Objects/71882""",NaN
2,"""https://example.com/ConXrefs/143182""","""https://example.com/Objects/78708""",NaN
3,"""https://example.com/ConXrefs/143517""","""https://example.com/Objects/78708""",NaN
4,"""https://example.com/ConXrefs/65968""","""https://example.com/Objects/41303""",NaN


In [18]:
import rdflib, os
from rdflib import Graph, URIRef
from rdflib.namespace import SKOS

def insert_links(df):
    g = Graph()

    try:
        for _ , row in df.iterrows():
            conxrefs = URIRef(str(row['s']).strip().replace('"', ''))
            object = URIRef(str(row['object']).strip().replace('"', ''))
            
            g.add((conxrefs, SKOS.related, object))
        
    finally:
        return g

In [5]:
df.columns

Index(['s', 'object', 'Unnamed: 2'], dtype='object')

In [20]:
insert_links(df).serialize(destination='conxreftoobject_graph.ttl')

<Graph identifier=N174cb54d847a4dfda0145e77ebbab74f (<class 'rdflib.graph.Graph'>)>